In [1]:
import numpy as np 
import pandas as pd 

In [2]:
trainset = pd.read_csv('./bigdata2021-rl-recsys/trainset.csv',' ')
trainset.head()

,user_id,user_click_history,user_protrait,exposed_items,labels,time
0,1,"30:1580603130,34:1581178937,15:1581178939,28:1...","64054,21804,80911,36504,8867,7615,54240,37294,...","1,20,28,99,86,119,213,237,164","1,1,1,1,1,1,1,1,1",1582992009
1,2,"20:1580644762,13:1580644765,1:1580644770,127:1...","64054,26565,93755,88510,6344,7615,54240,21927,...","1,4,26,112,86,117,191,234,172","1,1,1,1,1,1,1,1,0",1582992010
2,3,"39:1580772975,6:1580772981,1:1581178309,2:1581...","64054,64086,63021,88510,93500,7615,54240,21927...","22,4,28,48,105,42,193,236,159","1,1,1,0,1,0,0,0,0",1582992014
3,4,"6:1580657608,9:1580657612,15:1580657615,127:15...","64054,21531,6599,16721,37078,7615,54240,65505,...","5,16,1,74,133,122,235,218,172","1,1,1,1,1,1,1,1,0",1582992014
4,5,"37:1580743545,36:1580743554,25:1580743556,37:1...","64054,66036,6599,88510,76066,20543,83978,37294...","6,1,16,85,73,112,239,172,205","1,1,1,1,1,1,1,1,1",1582992017


In [24]:
trainset.shape

(260087, 11)

In [3]:
import time

In [4]:
start = time.time()
trainset['history_num'] = trainset["user_click_history"].apply(lambda row: list(map(lambda t: int(t.split(":")[0]), row.split(","))))
end = time.time()
trainset['time_line'] = trainset["user_click_history"].apply(lambda row: list(map(lambda t: int(t.split(":")[1]), row.split(","))))
trainset['reward'] = trainset["labels"].apply(lambda row: list(map(lambda t: 5*int(t), row.split(','))))
trainset['action'] = trainset["exposed_items"].apply(lambda row: list(map(lambda t: int(t), row.split(','))))
trainset['state'] = trainset['history_num'].apply(lambda row: row[-50:] if len(row) >= 50 else [0]*(50 - len(row)) + row)
print(end - start)

9.27688980102539


In [5]:
a = [0,0,0,0,0]
b = [5,1,2,5]
a[-len(b):] = b[-5:]
b = [5]*5 + b
b

[5, 5, 5, 5, 5, 5, 1, 2, 5]

In [11]:
trainset[trainset['labels'] == '1,1,1,1,1,1,0,0,1'].iloc[780,:]

user_id                                                           35126
user_click_history    14:1581787188,28:1581787191,6:1581787193,84:15...
user_protrait         64054,93918,38156,16721,37078,7615,54240,50138...
exposed_items                              2,7,18,107,51,81,239,190,172
labels                                                1,1,1,1,1,1,0,0,1
time                                                         1584206538
history_num           [14, 28, 6, 84, 55, 44, 164, 7, 15, 5, 6, 125,...
time_line             [1581787188, 1581787191, 1581787193, 158178719...
reward                                      [5, 5, 5, 5, 5, 5, 0, 0, 5]
action                           [2, 7, 18, 107, 51, 81, 239, 190, 172]
state                 [55, 44, 164, 7, 15, 5, 6, 125, 55, 101, 196, ...
Name: 35125, dtype: object

In [12]:
trainset[trainset['labels'] == '1,1,1,1,1,1,1,1,1'].iloc[0,:]

user_id                                                               1
user_click_history    30:1580603130,34:1581178937,15:1581178939,28:1...
user_protrait         64054,21804,80911,36504,8867,7615,54240,37294,...
exposed_items                             1,20,28,99,86,119,213,237,164
labels                                                1,1,1,1,1,1,1,1,1
time                                                         1582992009
history_num           [30, 34, 15, 28, 86, 132, 128, 172, 239, 35, 3...
time_line             [1580603130, 1581178937, 1581178939, 158117894...
reward                                      [5, 5, 5, 5, 5, 5, 5, 5, 5]
action                          [1, 20, 28, 99, 86, 119, 213, 237, 164]
state                 [16, 1, 72, 132, 100, 172, 239, 14, 18, 1, 133...
Name: 0, dtype: object

## create train

In [6]:
# train.to_csv('processed_train.csv')
# proc_train = pd.read_csv('processed_train.csv')
dic = {'state':trainset['state'], 'action':trainset['action'], 'reward':trainset['reward']}
train = pd.DataFrame(dic)
train.head()

,state,action,reward
0,"[16, 1, 72, 132, 100, 172, 239, 14, 18, 1, 133...","[1, 20, 28, 99, 86, 119, 213, 237, 164]","[5, 5, 5, 5, 5, 5, 5, 5, 5]"
1,"[29, 18, 107, 51, 133, 237, 213, 164, 19, 20, ...","[1, 4, 26, 112, 86, 117, 191, 234, 172]","[5, 5, 5, 5, 5, 5, 5, 5, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[22, 4, 28, 48, 105, 42, 193, 236, 159]","[5, 5, 5, 0, 5, 0, 0, 0, 0]"
3,"[56, 38, 6, 4, 102, 126, 53, 221, 237, 6, 7, 5...","[5, 16, 1, 74, 133, 122, 235, 218, 172]","[5, 5, 5, 5, 5, 5, 5, 5, 0]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 36, 25, 37,...","[6, 1, 16, 85, 73, 112, 239, 172, 205]","[5, 5, 5, 5, 5, 5, 5, 5, 5]"


In [6]:
# item_info = pd.read_csv('./bigdata2021-rl-recsys/item_info.csv',' ')
train.loc[0,'reward']

[5, 5, 5, 5, 5, 5, 5, 5, 5]

## create embed

In [7]:
item_info = pd.read_csv('./bigdata2021-rl-recsys/item_info.csv',' ')
for i in range(len(item_info)):
    item_info.loc[i, 'item_vec_num'] = item_info.loc[i, 'item_vec'] + ','+str(item_info.loc[i,'price']) + ',' + str(item_info.loc[i,'location'])
item_info.loc[0,'item_vec_num']

'2,2,1,0.015764,0.833,2954,1'

### turn it into array

In [8]:
item_info['item_vec_num'] = item_info['item_vec_num'].apply(lambda row: np.array(list(map(lambda t: float(t), row.split(',')))))
item_info.loc[0, 'item_vec_num']

array([2.0000e+00, 2.0000e+00, 1.0000e+00, 1.5764e-02, 8.3300e-01,
       2.9540e+03, 1.0000e+00])

In [9]:
#debug
item_info[item_info['item_id'] == 333]

,item_id,item_vec,price,location,item_vec_num
264,333,"1,0,2,0,0",2933,3,"[1.0, 0.0, 2.0, 0.0, 0.0, 2933.0, 3.0]"


### create dictionary

In [10]:
embed_vec = np.array(item_info['item_vec_num'].values.tolist(), dtype = float)
keys = [str(item) for item in item_info['item_id']]

In [11]:
from sklearn.preprocessing import MinMaxScaler
embed_vec = MinMaxScaler().fit_transform(embed_vec)
# sc.tranform(X_test)
embed_vec

array([[0.33333333, 0.22222222, 0.        , ..., 0.97884841, 0.1702386 ,
        0.        ],
       [0.33333333, 0.        , 0.        , ..., 0.91186839, 0.00176067,
        0.        ],
       [0.33333333, 0.88888889, 0.        , ..., 0.84723854, 0.03685265,
        0.        ],
       ...,
       [1.        , 0.11111111, 0.        , ..., 0.30199765, 0.08475502,
        1.        ],
       [1.        , 0.11111111, 0.        , ..., 0.48178613, 0.08487645,
        1.        ],
       [1.        , 0.11111111, 0.        , ..., 0.37955347, 0.08518001,
        1.        ]])

In [12]:
embed = dict(zip(keys, embed_vec))
embed['0'] = np.zeros([embed_vec.shape[1]])  #新增0元素
# embed

### add embeded columns for the environment data, finally we would only use the 'data.csv'

In [13]:
#debug
a = [1,2,3]
b = [0]
x = np.array(list(map(lambda key: embed[str(key)], a))).reshape(-1,)
y = np.array(list(map(lambda key: embed[str(key)], b))).reshape(-1,)
np.concatenate((x, y))  #the input of the network need to concat the state and action

array([3.33333333e-01, 2.22222222e-01, 0.00000000e+00, 1.63742707e-03,
       9.78848414e-01, 1.70238601e-01, 0.00000000e+00, 3.33333333e-01,
       0.00000000e+00, 0.00000000e+00, 1.30244215e-03, 9.11868390e-01,
       1.76067027e-03, 0.00000000e+00, 3.33333333e-01, 8.88888889e-01,
       0.00000000e+00, 7.80904385e-04, 8.47238543e-01, 3.68526501e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [36]:
data = pd.DataFrame()
data['embeded_state'] = train['state'].apply(lambda row: np.array(list(map(lambda key: embed[str(key)], row))).reshape(-1,))
data['state'] = train['state']
data['embeded_action'] = train['action'].apply(lambda row: np.array(list(map(lambda key: embed[str(key)], row))))  # 0 for the first action
data['action'] = train['action']
data['reward'] = train['reward']

In [35]:
#debug


In [37]:
data['del_zero'] = data['state'].apply(lambda row: np.sum(row))
data = data[data['del_zero'] != 0].drop(columns = ['del_zero'])
data

,embeded_state,state,embeded_action,action,reward
0,"[0.6666666666666667, 0.0, 0.0, 0.0007182699945...","[16, 1, 72, 132, 100, 172, 239, 14, 18, 1, 133...","[[0.3333333333333333, 0.2222222222222222, 0.0,...","[1, 20, 28, 99, 86, 119, 213, 237, 164]","[5, 5, 5, 5, 5, 5, 5, 5, 5]"
1,"[1.0, 0.2222222222222222, 0.0, 0.0017837817232...","[29, 18, 107, 51, 133, 237, 213, 164, 19, 20, ...","[[0.3333333333333333, 0.2222222222222222, 0.0,...","[1, 4, 26, 112, 86, 117, 191, 234, 172]","[5, 5, 5, 5, 5, 5, 5, 5, 0]"
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.6666666666666667, 0.0, 0.0, 0.000312237108...","[22, 4, 28, 48, 105, 42, 193, 236, 159]","[5, 5, 5, 0, 5, 0, 0, 0, 0]"
3,"[0.0, 0.6666666666666666, 0.0, 0.0007435107188...","[56, 38, 6, 4, 102, 126, 53, 221, 237, 6, 7, 5...","[[0.3333333333333333, 0.0, 0.0, 0.001603253415...","[5, 16, 1, 74, 133, 122, 235, 218, 172]","[5, 5, 5, 5, 5, 5, 5, 5, 0]"
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 36, 25, 37,...","[[0.3333333333333333, 0.7777777777777777, 0.0,...","[6, 1, 16, 85, 73, 112, 239, 172, 205]","[5, 5, 5, 5, 5, 5, 5, 5, 5]"
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.777...","[0, 34, 39, 29, 16, 28, 32, 72, 127, 132, 238,...","[[0.3333333333333333, 0.8888888888888888, 0.0,...","[3, 19, 5, 99, 83, 92, 164, 172, 183]","[5, 5, 5, 5, 5, 5, 5, 5, 5]"
6,"[1.0, 0.0, 0.0, 0.0015296086680178936, 0.81786...","[32, 32, 28, 36, 80, 51, 53, 221, 237, 183, 32...","[[0.3333333333333333, 0.0, 0.0, 0.000615125800...","[8, 20, 1, 72, 100, 107, 233, 188, 164]","[5, 5, 5, 5, 5, 5, 5, 5, 5]"
7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[1.0, 0.8888888888888888, 0.0, 0.000712037716...","[30, 27, 5, 53, 60, 103, 164, 185, 157]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 7, 20, 33, 66, 85, 53, 172,...","[[0.6666666666666667, 0.2222222222222222, 0.0,...","[15, 33, 10, 41, 48, 137, 150, 158, 233]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,"[0.6666666666666667, 0.2222222222222222, 0.0, ...","[15, 19, 34, 127, 86, 104, 127, 217, 247, 19, ...","[[0.6666666666666667, 0.2222222222222222, 0.0,...","[15, 32, 34, 51, 110, 78, 240, 237, 172]","[5, 5, 5, 0, 0, 0, 0, 0, 0]"


In [36]:
l = data.loc[0,'state']

In [16]:
# data = pd.DataFrame()
# data = data.assign(embed_state = train['embeded_state'], embed_action = train['embeded_action'], reward = train['reward'])
data.iloc[0,1]

array([3.33333333e-01, 2.22222222e-01, 0.00000000e+00, 1.63742707e-03,
       9.78848414e-01, 1.70238601e-01, 0.00000000e+00, 6.66666667e-01,
       7.77777778e-01, 0.00000000e+00, 6.32991663e-04, 9.17743831e-01,
       1.10982940e-01, 0.00000000e+00, 1.00000000e+00, 4.44444444e-01,
       0.00000000e+00, 2.63698053e-03, 1.00000000e+00, 8.80335134e-03,
       0.00000000e+00, 6.66666667e-01, 1.11111111e-01, 0.00000000e+00,
       5.96844452e-04, 7.73207991e-01, 2.48315221e-02, 5.00000000e-01,
       3.33333333e-01, 6.66666667e-01, 0.00000000e+00, 1.26837236e-03,
       8.98942421e-01, 9.97510777e-02, 5.00000000e-01, 6.66666667e-01,
       1.11111111e-01, 0.00000000e+00, 4.16523887e-05, 6.68625147e-01,
       7.76516301e-02, 5.00000000e-01, 6.66666667e-01, 2.22222222e-01,
       0.00000000e+00, 8.63274320e-04, 4.75910693e-01, 1.24400461e-01,
       1.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       8.70441439e-04, 4.39482961e-01, 3.27848947e-02, 1.00000000e+00,
      

In [23]:
data.loc[0,'embeded_action']

array([[3.33333333e-01, 2.22222222e-01, 0.00000000e+00, 1.63742707e-03,
        9.78848414e-01, 1.70238601e-01, 0.00000000e+00],
       [6.66666667e-01, 7.77777778e-01, 0.00000000e+00, 6.32991663e-04,
        9.17743831e-01, 1.10982940e-01, 0.00000000e+00],
       [1.00000000e+00, 4.44444444e-01, 0.00000000e+00, 2.63698053e-03,
        1.00000000e+00, 8.80335134e-03, 0.00000000e+00],
       [6.66666667e-01, 1.11111111e-01, 0.00000000e+00, 5.96844452e-04,
        7.73207991e-01, 2.48315221e-02, 5.00000000e-01],
       [3.33333333e-01, 6.66666667e-01, 0.00000000e+00, 1.26837236e-03,
        8.98942421e-01, 9.97510777e-02, 5.00000000e-01],
       [6.66666667e-01, 1.11111111e-01, 0.00000000e+00, 4.16523887e-05,
        6.68625147e-01, 7.76516301e-02, 5.00000000e-01],
       [6.66666667e-01, 2.22222222e-01, 0.00000000e+00, 8.63274320e-04,
        4.75910693e-01, 1.24400461e-01, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.70441439e-04,
        4.39482961e-01, 3

In [19]:
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260087 entries, 0 to 260086
Data columns (total 3 columns):
embeded_state     260087 non-null object
embeded_action    260087 non-null object
reward            260087 non-null object
dtypes: object(3)
memory usage: 216.3 MB


In [13]:
from sklearn.utils import shuffle
def train_test_split(data, train_size = 0.7, shuffle_data = True, random_seed = None):
    if shuffle_data == True:
        data = shuffle(data, random_state = random_seed)
    ind = int(len(data)*train_size)
    train_data = data[0:ind].reset_index(drop = True)
    test_data = data[ind:].reset_index(drop = True)
    return train_data, test_data

train_data, test_data = train_test_split(data, random_seed = 1)
len(train_data) + len(test_data)

260087

In [15]:
data['embeded_state'].head()

0    [0.6666666666666667, 0.0, 0.0, 0.0007182699945...
1    [1.0, 0.2222222222222222, 0.0, 0.0017837817232...
2    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0,...
3    [0.0, 0.6666666666666666, 0.0, 0.0007435107188...
4    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0,...
Name: embeded_state, dtype: object

In [24]:
# from concurrent.futures import ProcessPoolExecutor
aa={'one':['1','2','1','2'],'two':[np.ones(3),np.zeros(3),np.zeros(3)+1.5,np.ones(3)],'three':[3,8,5,8]}
aa = pd.DataFrame(aa)
aa

,one,two,three
0,1,"[1.0, 1.0, 1.0]",3
1,2,"[0.0, 0.0, 0.0]",8
2,1,"[1.5, 1.5, 1.5]",5
3,2,"[1.0, 1.0, 1.0]",8
